# README
Generic testing document.

In [1]:
print("Importing torch...")
import torch
import torch.nn as nn
print("Importing HF...")
from transformers import AutoTokenizer, AutoModelForCausalLM
print("Importing python modules...")
from timehelp import time_start, time_end
from model_wrapper import Model, ModelFamily, MultipleChoiceStrategy
import re
print("Done!")

Importing torch...
Importing HF...
Importing python modules...
Done!


In [2]:
# options: 350M, 2B, 6B, 16B
model = Model(ModelFamily.CodeGen1.mono["2B"])
model.configure(time=True)

[2024-05-06@17:47:00|model.device] Starting timer.
Configuring torch device...
Using device: cuda:0 aka cuda:0
[2024-05-06@17:47:00|model.device] Time elapsed: 44ms
[2024-05-06@17:47:00|model.tokenizer] Starting timer.
[2024-05-06@17:47:00|model.tokenizer] Time elapsed: 249ms
[2024-05-06@17:47:00|model.model] Starting timer.
Obtaining model...
[2024-05-06@17:47:34|model.model] Time elapsed: 33s 327ms


In [3]:
prompt = """Python code:
for i in range(23):
    print(i)

  choice: prints 'hello world' to the terminal
  choice: prints values from 1 to 10
  choice: computes first 10 prime numbers
  choice: prints values from 0 to 22

English language description:"""

targets = re.findall(r"choice: (.+)", prompt)

print("Choices: ", targets)

idx = model.multiple_choice_prompts(
    prompt, targets,
    time=False,
    strategy=MultipleChoiceStrategy.MULTIPLY
)
print(idx)

# 350M FIRST_BRANCH: 2
# 350M LOGIT_AVERAGE: 1
# 350M MULTIPLY: 3

Choices:  ["prints 'hello world' to the terminal", 'prints values from 1 to 10', 'computes first 10 prime numbers', 'prints values from 0 to 22']
Tokenizing input prompt...
Token count in input: 84
Token count in input: 8
Token count in input: 6
Token count in input: 6
Token count in input: 6
Token 1: P=0.0008649470400996506, logit=28.836490631103516
Token 2: P=2.8487263534771046e-06, logit=116.56971740722656
Token 3: P=3.525448846630752e-05, logit=119.57723236083984
Token 4: P=4.0011909732129425e-05, logit=109.87080383300781
Token 5: P=3.900055162375793e-05, logit=120.9642105102539
Token 6: P=0.00010875055886572227, logit=109.5257797241211
Token 7: P=2.342500738450326e-05, logit=118.17300415039062
Token 1: P=0.00035107971052639186, logit=119.63996887207031
Token 2: P=0.0002885882568079978, logit=121.21388244628906
Token 3: P=6.494770059362054e-05, logit=121.79008483886719
Token 4: P=3.481068051769398e-05, logit=120.9379653930664
Token 5: P=7.137858574424172e-06, logit=115.532569885253

In [4]:
prompt = """Python code:
for i in range(23):
    print(i)

  choice: prints 'hello world' to the terminal
  choice: prints values from 1 to 10
  choice: computes first 10 prime numbers
  choice: prints values from 0 to 22

English language description:"""

targets = re.findall(r"choice: (.+)", prompt)

print("Choices: ", targets)

# target_tokens = [
#     (idx, model.tokenize(target)["input_ids"])
#     for idx, target in enumerate(targets)
# ]

inputs = model.tokenize(prompt)

with torch.no_grad():
    base_logits = None
    output = model.model(input_ids=inputs["input_ids"])
    base_logits = output.logits[:, -1, :]

    best_score = float("-inf")
    best_option = None
    
    for target in targets:
        print("Target:", target)
        target_tokens = model.tokenize(target)["input_ids"]
        print("Base score:", base_logits[:, target_tokens[0, 0]])
        score = base_logits[:, target_tokens[0, 0]].item()
        running_inputs = inputs["input_ids"]

        print("initial score =", score)
        
        for idx in range(1, target_tokens.shape[1]):
            token = target_tokens[0, idx]
            token_formatted = token.unsqueeze(0).unsqueeze(0)
            running_inputs = torch.cat((running_inputs, token_formatted), dim=-1)
            output = model.model(input_ids=running_inputs)
            next_logits = output.logits[:, -1, :]
            print("Inner score:", next_logits[:, token])
            score += next_logits[:, token].item()

        print("Final score =", score)
        # does this even work
        score /= target_tokens.shape[1]
        print("Normalized =", score)
        
        if best_option is None or score > best_score:
            best_score = score
            best_option = target

        print()
        
    # if running_tensor is None:
    #     running_tensor = logits
    # else:
    #     running_tensor += logits

print("Best:", best_option, best_score)


if False:
    choice_idx, choice_tokens = model.multiple_choice_prompts(prompt, targets=choices)
    print("Most likely:", choices[choice_idx])

Choices:  ["prints 'hello world' to the terminal", 'prints values from 1 to 10', 'computes first 10 prime numbers', 'prints values from 0 to 22']
Token count in input: 84
Target: prints 'hello world' to the terminal
Token count in input: 8
Base score: tensor([4.5035], device='cuda:0')
initial score = 4.503489017486572
Inner score: tensor([6.8510], device='cuda:0')
Inner score: tensor([9.0938], device='cuda:0')
Inner score: tensor([12.9985], device='cuda:0')
Inner score: tensor([9.5918], device='cuda:0')
Inner score: tensor([8.4253], device='cuda:0')
Inner score: tensor([9.6558], device='cuda:0')
Inner score: tensor([11.3185], device='cuda:0')
Final score = 72.43814039230347
Normalized = 9.054767549037933

Target: prints values from 1 to 10
Token count in input: 6
Base score: tensor([4.5035], device='cuda:0')
initial score = 4.503489017486572
Inner score: tensor([9.7278], device='cuda:0')
Inner score: tensor([12.4518], device='cuda:0')
Inner score: tensor([13.8390], device='cuda:0')
Inn

In [17]:
model.multiple_choice_token("( False ) is ", ["True", "False"], time=True)

Tokenizing input prompt...
[2024-03-29@20:01:17|model.tokenize] Starting timer.
[2024-03-29@20:01:17|model.tokenize] Time elapsed: 2ms
Token count in input: 5


(0, 'True')

In [7]:
inputs = model.tokenize("def hello_world():")

Token count in input: 5


In [9]:
## FREE RESPONSE ##
sample = model.generate(inputs, time=True, max_new_tokens=64)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating...
[2024-03-29@19:52:39|model.generate] Starting timer.
[2024-03-29@19:52:42|model.generate] Time elapsed: 2s 568ms


In [11]:
print(model.decode(sample, inputs))


    print("Hello World")

hello_world()

# 파이썬의 내장 함수
# 파이썬의 내장 함수


In [138]:
def query_multiple_choice(model, inputs, targets):
    input_tokens = tokenizer(inputs, return_tensors="pt").to(use_device)
    
    with torch.no_grad():
        output = model(input_ids=input_tokens["input_ids"])
        logits = output.logits[:, -1, :]

    target_ids = tokenizer.convert_tokens_to_ids(targets)
    subset_logits = logits[:, target_ids]
    predicted_idx = torch.argmax(subset_logits, dim=-1).item()
    predicted_token = targets[predicted_idx]

    return predicted_idx, predicted_token

In [ ]:
time_start()
# print(tokenizer.decode(sample[0], truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"]))
print(tokenizer.decode(sample[:, inputs["input_ids"].shape[1]:][0]))
# print(tokenizer.decode(sample[0]))
time_end()